## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/theon/School/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ushuaia,AR,-54.80,-68.30,50.00,57,75,11.41,broken clouds
1,1,Laguna,US,38.42,-121.42,86.00,28,1,2.91,clear sky
2,2,Hermanus,ZA,-34.42,19.23,52.00,95,0,8.23,clear sky
3,3,Bluff,NZ,-46.60,168.33,64.99,50,86,3.00,overcast clouds
4,4,Georgetown,MY,5.41,100.34,77.00,83,20,5.82,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
weather_preference_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
weather_preference_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ushuaia,AR,-54.80,-68.30,50.00,57,75,11.41,broken clouds
2,2,Hermanus,ZA,-34.42,19.23,52.00,95,0,8.23,clear sky
3,3,Bluff,NZ,-46.60,168.33,64.99,50,86,3.00,overcast clouds
4,4,Georgetown,MY,5.41,100.34,77.00,83,20,5.82,few clouds
5,5,Matara,LK,5.95,80.54,78.60,86,5,10.18,light rain
6,6,Faanui,PF,-16.48,-151.75,79.11,76,5,11.27,clear sky
8,8,Kaitangata,NZ,-46.28,169.85,59.63,49,98,7.11,overcast clouds
10,10,Rikitea,PF,-23.12,-134.97,74.46,71,100,4.09,overcast clouds
11,11,Vaini,TO,-21.20,-175.20,75.20,73,75,10.29,broken clouds
15,15,East London,ZA,-33.02,27.91,63.18,81,0,12.10,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
weather_preference_df.count()

City_ID        1087
City           1087
Country        1074
Lat            1087
Lng            1087
Max Temp       1087
Humidity       1087
Cloudiness     1087
Wind Speed     1087
Description    1087
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# The cities in the database within the country Namibia were incorrectly converted to 'NaN' due to having the country code 'NA'
# This code is to correct that
weather_preference_df["Country"].fillna("NM", inplace = True)

C:\Users\theon\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [7]:
weather_preference_df.count()

City_ID        1087
City           1087
Country        1087
Lat            1087
Lng            1087
Max Temp       1087
Humidity       1087
Cloudiness     1087
Wind Speed     1087
Description    1087
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = weather_preference_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Ushuaia,AR,50.00,broken clouds,-54.80,-68.30,
2,Hermanus,ZA,52.00,clear sky,-34.42,19.23,
3,Bluff,NZ,64.99,overcast clouds,-46.60,168.33,
4,Georgetown,MY,77.00,few clouds,5.41,100.34,
5,Matara,LK,78.60,light rain,5.95,80.54,
6,Faanui,PF,79.11,clear sky,-16.48,-151.75,
8,Kaitangata,NZ,59.63,overcast clouds,-46.28,169.85,
10,Rikitea,PF,74.46,overcast clouds,-23.12,-134.97,
11,Vaini,TO,75.20,broken clouds,-21.20,-175.20,
15,East London,ZA,63.18,clear sky,-33.02,27.91,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")            

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Ushuaia,AR,50.00,broken clouds,-54.80,-68.30,Albatros Hotel
2,Hermanus,ZA,52.00,clear sky,-34.42,19.23,Misty Waves Boutique Hotel
3,Bluff,NZ,64.99,overcast clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
4,Georgetown,MY,77.00,few clouds,5.41,100.34,Cititel Penang
5,Matara,LK,78.60,light rain,5.95,80.54,Amaloh Boutique Resort
6,Faanui,PF,79.11,clear sky,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
8,Kaitangata,NZ,59.63,overcast clouds,-46.28,169.85,Kaitangata Motor Camp
10,Rikitea,PF,74.46,overcast clouds,-23.12,-134.97,Pension Maro'i
11,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort
15,East London,ZA,63.18,clear sky,-33.02,27.91,Tu Casa


In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np 

hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Ushuaia,AR,50.00,broken clouds,-54.80,-68.30,Albatros Hotel
2,Hermanus,ZA,52.00,clear sky,-34.42,19.23,Misty Waves Boutique Hotel
3,Bluff,NZ,64.99,overcast clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
4,Georgetown,MY,77.00,few clouds,5.41,100.34,Cititel Penang
5,Matara,LK,78.60,light rain,5.95,80.54,Amaloh Boutique Resort
...,...,...,...,...,...,...,...
1889,Ponnani,IN,73.40,mist,10.77,75.93,Rouba Residency
1890,Maler Kotla,IN,66.27,clear sky,30.52,75.88,Milan editing point
1891,Pontes E Lacerda,BR,77.54,clear sky,-15.23,-59.34,Hotel Bressan
1892,Coupar Angus,GB,50.00,heavy intensity rain,56.55,-3.27,Easter Bendochy House


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/theon/School/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))